## Stat 135 lab 1 

Author: Zihan Wen

In [1]:
#first import all the libraries needed 
import numpy as np 
import random
import plotly.figure_factory as ff
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pip install plotly-geo

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade pip

Requirement already up-to-date: pip in /srv/app/venv/lib/python3.6/site-packages (20.1.1)
Note: you may need to restart the kernel to use updated packages.


## Q1

**Used R to save the files as csv file and used pandas to read them into dataframes**

In [4]:
#used R to save the files as csv file and used pandas to read them into dataframes
data = pd.read_csv('radon.csv')
data = data.drop('Unnamed: 0', axis = 1)

sizes = pd.read_csv('sizes.csv', header = 1)
sizes = sizes.drop('1', axis = 1)

In [5]:
data.head()

,county,radon
0,1,1.0
1,1,2.2
2,1,2.2
3,1,2.9
4,2,2.4


**ratio : how many houses does each sample represent in each county (the population column in the dataframe is in 100s)**

In [6]:
#ratio : population/sample (the population column in the dataframe is in 100s)
sizes['ratio'] = sizes['population'] / sizes['sample'] *100
sizes.head()

,county,sample,population,ratio
0,1,4,54,1350.000000
1,2,57,719,1261.403509
2,3,4,110,2750.000000
3,4,7,115,1642.857143
4,5,4,95,2375.000000


In [7]:
#there are 443 rows with 'radon' > 4
filtered = data[data['radon'] >4 ]
filtered.shape

(443, 2)

**To estimate the population proportion, first calculate out the sample proportion. To do that, first count the number of samples in each county with radon level above 4. With that, find how many houses does one sample represent in each county and multiply the ratio with the county counts, which should give us a sample sum. Finally, divide the estimated sum by the total population, which should give us the estimated proportion from our sample.**

In [8]:
value_count = filtered['county'].value_counts().to_frame()
value_count = value_count.rename(columns = {"county" : "sample_count"})
value_count.head()

merged = pd.merge(value_count, sizes, how = 'left', left_index = True, right_on = 'county')
merged = merged.set_index('county')

series = merged['sample_count'] * merged['ratio']
sample_sum = sum(series.values)
population_sum = sum(sizes['population']) * 100
estimated_proportion = sample_sum / population_sum
estimated_proportion

0.47097494727582057

## Thus, the estimated proportion is 0.4710

In [9]:
sample_size = sum(sizes['sample'])
sample_size

1003

**Use the formula var(p_hat) = sum(w_i ** 2 * sigma_i ** 2 / n_i *( Ni - ni)/ (N_i - 1)))to calculate the standard error**

In [32]:
merged['sample_prop'] = merged['sample_count']  * merged['ratio'] / (merged['population'] * 100)
merged.head()

,sample_count,sample,population,ratio,sample_prop
county,,,,,
27,56,119,3925,3298.319328,0.470588
19,34,69,794,1150.724638,0.492754
82,24,50,424,848.000000,0.480000
69,23,122,81,66.393443,0.188525
2,17,57,719,1261.403509,0.298246


In [44]:
w = merged['population'] * 100 / population_sum
sigma_squared = merged['sample_prop'] * (1 - merged['sample_prop']) / merged['sample']
finite_pop_corr = (merged['population'] * 100 - merged['sample']) / (merged['population'] * 100 - 1)
np.sqrt(sum(w ** 2 * sigma_squared * finite_pop_corr))

0.018259738558231924

## Thus, the estimaed standard error is approximately 0.0183

## Q2

In [12]:
county_few_samples = sizes[sizes['sample'] == 4]
county_few_samples

,county,sample,population,ratio
0,1,4,54,1350.0
2,3,4,110,2750.0
4,5,4,95,2375.0
7,8,4,102,2550.0
14,15,4,31,775.0
16,17,4,52,1300.0
29,30,4,90,2250.0
32,33,4,49,1225.0
33,34,4,141,3525.0
40,41,4,29,725.0


**Let's pick county 3!**

In [13]:
three = county_few_samples[county_few_samples['county'] == 3]
three

,county,sample,population,ratio
2,3,4,110,2750.0


In [14]:
#There are two samples with radon > 4 in county 3
filtered[filtered['county'] == 3]

,county,radon
62,3,4.3
63,3,4.5


In [15]:
#neighboring counties of county 1
neighbors = [3, 14, 56, 80, 29, 15, 44]
neighbors_data = filtered[filtered['county'].isin(neighbors)]
n_value_counts = neighbors_data['county'].value_counts().to_frame()
n_value_counts = n_value_counts.rename(columns = {'county' : 'sample_count'})
n_value_counts

,sample_count
14,9
56,4
44,3
3,2
29,1
80,1
15,1


In [16]:
neighbor_merged = pd.merge(n_value_counts, sizes, how = 'left', left_index = True, right_on = 'county')
neighbor_merged = neighbor_merged.set_index('county')
neighbor_merged

,sample_count,sample,population,ratio
county,,,,
14,9,15,172,1146.666667
56,4,11,199,1809.090909
44,3,9,45,500.000000
3,2,4,110,2750.000000
29,1,5,57,1140.000000
80,1,5,50,1000.000000
15,1,4,31,775.000000


In [17]:
n_series = neighbor_merged['sample_count'] * neighbor_merged['ratio']
n_sample_sum = sum(n_series.values)
n_population_sum = sum(neighbor_merged['population']) * 100
n_estimated_proportion = n_sample_sum / n_population_sum
n_estimated_proportion

0.41372535596933185

**Thus, with our new procedure 1 (use the estimated proportions from neighboring counties to estimate the proportion of county of interest), the estimated proportion for county 3 is 0.4137.**

**Procure 2: Bootstrapping**

In [18]:
one = sum(neighbor_merged['sample_count'])
zero  = sum(neighbor_merged['sample']) - one

pop  = list(np.ones(one))+list(np.zeros(zero))
random.sample(pop, 4)

[0.0, 1.0, 0.0, 0.0]

In [19]:
arr = []
for i in np.arange(10000):
    arr += random.sample(pop, 4)
est_prop_2 = sum(arr) / (10000 * 4)
est_prop_2

0.39355

**Procedure 2 gives us 0.39135 as the estimated proportion for county 3.**

**I chose procedure 2 over procedure 1 since procedure one relies on the samples from neighboring states only. This might leads to higher bias.**

## 2 b)

In [20]:
merged.head()

,sample_count,sample,population,ratio
county,,,,
27,56,119,3925,3298.319328
19,34,69,794,1150.724638
82,24,50,424,848.000000
69,23,122,81,66.393443
2,17,57,719,1261.403509


In [21]:
#the counties that do not appear in this series have estimated proportion of 0
est_pop_counties = merged['sample_count']  * merged['ratio'] / (merged['population'] * 100).sort_values()
est_pop_counties

county
2     0.298246
3     0.500000
4     0.571429
5     0.250000
6     0.666667
7     0.785714
8     0.500000
9     0.363636
10    0.666667
11    0.400000
12    0.600000
13    0.500000
14    0.600000
15    0.250000
17    0.500000
18    0.416667
19    0.492754
20    1.000000
21    0.818182
22    0.333333
24    0.700000
25    0.666667
27    0.470588
28    0.500000
29    0.200000
31    0.250000
32    0.857143
33    0.250000
34    1.000000
35    0.666667
        ...   
54    0.333333
55    0.384615
56    0.363636
57    0.250000
58    0.166667
59    0.750000
60    0.500000
62    0.357143
64    0.600000
65    0.666667
66    0.818182
67    0.666667
68    0.357143
69    0.188525
70    0.571429
72    0.750000
73    0.555556
74    0.600000
75    1.000000
76    0.250000
77    0.500000
78    0.400000
79    0.714286
80    0.200000
81    0.250000
82    0.480000
83    1.000000
84    1.000000
85    0.692308
86    0.714286
Length: 77, dtype: float64

In [26]:
#try to do it in Python
fips = [27000 + (2*i-1) for i in est_pop_counties.index.tolist()] + [27001, 27031, 27045, 27051,27059, 27061, 27085, 27121, 27125, 27135, 27141, 27173]
values = est_pop_counties.values.tolist() + [0,0,0,0,0,0,0,0,0,0,0,0]
state = ['Minnesota']
binning_endpoints=[0.2, 0.4, 0.6, 0.8]
colorscale = colorscale = [
    'rgb(193, 193, 193)',
    'rgb(239,239,239)',
    'rgb(195, 196, 222)',
    'rgb(144,148,194)',
    'rgb(101,104,168)'
]

fig = ff.create_choropleth(
    fips=fips, values=values, scope=state, show_state_data=True, county_outline={'color': 'rgb(255,255,255)', 'width': 0.5}, binning_endpoints=binning_endpoints, colorscale=colorscale, title = "Proportion of houses with Radon level above 4pCi/l"
)
fig.layout.template = None
fig.show()